In [1]:
import json
import requests
import popelines
import os
from datetime import datetime, timedelta
from main import fix_values, populating_vals, flatten, flatten_dupe_vals
    

In [2]:
from generate_original_with_timeseries import read_endpoint, generate_results

In [3]:
# ############################################### Keys and Authentication #######################################
# if not os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS'):
#     os.environ['GOOGLE_ACCOUNT_CREDENTIALS'] = '/home/engineering/keyfile.json'
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

# directory = str(os.path.abspath(os.path.dirname(__file__)))
directory = os.getcwd()

############################################### Instantiating Popelines #######################################
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)


In [4]:
# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}


In [5]:
params = (
    ('per_page', 100),
) 

In [6]:
query = open(f'{directory}/existing_experiments.sql').read()

In [22]:
def results_generator(start_timestamp, end_timestamp, experiment_id):
    now = datetime.strptime(start_timestamp, '%Y-%m-%dT%H:%M:%SZ')
    exp_results = []
    
    while now <= datetime.strptime(end_timestamp,'%Y-%m-%dT%H:%M:%SZ'):
        start_ts = datetime.strftime(now, '%Y-%m-%dT%H:%M:%SZ')
        end_ts = datetime.strftime(now + timedelta(hours=1), '%Y-%m-%dT%H:%M:%SZ')
        params = (
            ('per_page', 100),
        ) 

        result_endpoint = f"https://api.optimizely.com/v2/experiments/{experiment_id}/results?start_time={start_ts}&end_time={end_ts}"
        response_ts = read_endpoint(result_endpoint, headers, params)
        print(f"got experiment {experiment_id}")

        # if '' then the experiment has not started yet
        if response_ts == '' or 'bad' in response_ts:
            j_ts = {'experiment_id': experiment_id}
            new_j_ts = j_ts
            new_j_ts['upload_ts'] = str(datetime.utcnow())
            new_j_ts['metric_calculating_ts'] = end_ts
            exp_results.extend(new_j_ts)

        else:

            new_j_ts = pope.fix_json_values(callback=fix_values, obj=response_ts, reset_key='results')
            newer_j_ts = pope.fix_json_values(callback=fix_values, obj=new_j_ts, reset_key='variations')
            flattened_j_ts = generate_results(newer_j_ts)
            exp_results.extend(flattened_j_ts)

        now += timedelta(hours=1)
        
    return exp_results

# testing experiment status

In [23]:
results = pope.bq_query(query)

In [26]:
all_exp = []
for result in results:
    details = {}
    details['id'] = result['id']
    if result['earliest'] is None:
        details['earliest'] = ''
        details['status'] = 'experiment_archived'
    else:
        details['earliest'] = result['earliest']
        details['status'] = result['status']
    details['last_modified'] = result['last_modified']
    all_exp.append(details)

In [27]:
for exp in all_exp:
    if exp['status'] == 'archived' or exp['status'] == 'experiment_archived':
        
        # original:
        start_timestamp = datetime.strftime(exp['earliest'], '%Y-%m-%dT%H:%M:%SZ')[:14] + str('00:00Z')
        end_timestamp = datetime.strftime(exp['last_modified'], '%Y-%m-%dT%H:%M:%SZ')[:14] + str('00:00Z')
        exp_results = results_generator(start_timestamp, end_timestamp, experiment_id=exp['id'])

        # chunk up time lapse by hour, make API calls
        # no need to update
    elif exp['status'] == 'not_started':
        # original: do not append
        print('no results to be backfilled')
        pass
        # update:
        # each hour, check status change
    #     update_status = NewStatus
    elif exp['status'] == 'running':
        # original:
        start_timestamp = exp['earliest'][:14] + str('00:00Z')
        end_timestamp = datetime.strftime(datetime.now(), '%Y-%m-%dT%H:%M:%SZ')
        exp_results = results_generator(start_timestamp, end_timestamp, experiment_id=exp['id'])

        # update:
        # each hour, check status change
    #     update_status = NewStatus
    elif exp['status'] == 'paused':
        # original:
        start_timestamp = exp['earliest']
        end_timestamp = exp['last_modified']
        # update:
    #     update_status = NewStatus
    else:
        print(f'Experiment {exp['id']} shows a new experiment status. Need to investigate.')



got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
got experiment 10799420505
g

In [28]:
exp_results

[{'confidence_threshold': 0.9,
  'end_time': '2018-05-30T22:00:00.000000Z',
  'experiment_id': 10799420505,
  'metrics_aggregator': 'unique',
  'metrics_event_id': 10754570776,
  'metrics_name': 'Marketo Demo Submit',
  'metrics_results_is_baseline': False,
  'metrics_results_level': 'variation',
  'metrics_results_lift_is_significant': False,
  'metrics_results_lift_lift_status': 'equal',
  'metrics_results_lift_significance': 0.0,
  'metrics_results_lift_value': 0.0,
  'metrics_results_lift_visitors_remaining': 9223372036854775807,
  'metrics_results_name': 'Smaller Images - No Chat',
  'metrics_results_rate': 0.0,
  'metrics_results_results_id': '10799731267',
  'metrics_results_samples': 35,
  'metrics_results_upload_ts': '2019-03-01 23:12:32.929274',
  'metrics_results_value': 0.0,
  'metrics_results_variance': 0.0,
  'metrics_results_variation_id': '10799731267',
  'metrics_scope': 'visitor',
  'metrics_winning_direction': 'increasing',
  'reach_baseline_count': 32,
  'reach_base

In [29]:
pope.write_to_json(file_name=f'{directory}/../uploads/testing_results.json', jayson=exp_results, mode='w')

In [30]:
pope.write_to_bq(table_name='testing_results', file_name=f'{directory}/../uploads/testing_results.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)    


INFO:root:Writing testing_results to BQ from file /Users/vicky/Dev/optimizely/scripts/../uploads/testing_results.json


2019-03-01 15:16:32,596 - root - Writing testing_results to BQ from file /Users/vicky/Dev/optimizely/scripts/../uploads/testing_results.json
